In [1]:

import sys
sys.path.append('pizza/utils')
from trees import TopSemanticTree, ExpressSemanticTree
from semantic_matchers import is_unordered_exact_match, is_semantics_only_unordered_exact_match, \
                              is_semantics_only_unordered_exact_match_post_ER, is_unordered_exact_match_post_ER, is_semantics_only_unordered_exact_match_post_ER_top_top
from entity_resolution import PizzaSkillEntityResolver
import json
import csv

In [2]:
top_pred_string_same = "(ORDER (PIZZAORDER (NUMBER duck ) (SIZE medium ) (TOPPING tomatoes ) (TOPPING hammmmmm ) nnn ) )"
exr_string = "(ORDER (PIZZAORDER (TOPPING cheese ) (SIZE mediu ) (TOPPING HAM ) (TOPPING TOMATOES ) nnn ) )"

In [3]:
resolver = PizzaSkillEntityResolver()
print (is_semantics_only_unordered_exact_match_post_ER_top_top(top_pred_string_same, exr_string, resolver))

0.6666666666666667


In [4]:
import json

def load_data(file_path, num_lines):
    data = []
    with open(file_path, 'r') as file:
        for _ in range(num_lines):
            data.append(json.loads(file.readline()))
    return data  # Explicitly return the data

data = load_data('../PIZZA_dev.json', 847)

def getextensions(datatype):
    if datatype == 'train':
        return 'train.SRC', 'train.EXR', 'train.TOP'
    elif datatype == 'dev':
        return 'dev.SRC', 'dev.EXR', 'dev.TOP'
    else:
        return 'test.SRC', 'test.EXR', 'test.TOP'

def get_training_data(data, datatype='dev'):
    values = getextensions(datatype)
    training_data = []
    training_exr = []
    training_top = []
    # training_top_dec = []
    for item in data:
        training_data.append(item[values[0]])
        training_exr.append(item[values[1]])
        training_top.append(item[values[2]])
        # if datatype == 'train':
        #     training_top_dec.append(item[values[3]])
    return training_data, training_exr, training_top

training_data, training_exr, training_top = get_training_data(data)
print(training_exr[0])

(ORDER (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING CHEESE ) ) (TOPPING PEPPERONI ) ) (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (TOPPING OLIVES ) (TOPPING SAUSAGE ) ) (PIZZAORDER (NUMBER 3 ) (SIZE LARGE ) (TOPPING PEPPERONI ) (TOPPING SAUSAGE ) ) )


In [5]:
import inflect
import re
from word2number import w2n

def words_to_numbers(text):
    # Initialize inflect engine
    p = inflect.engine()
    
    # Function to convert matches to numbers
    def word_to_number(match):
        try:
            # Get the matched word and remove underscores
            word = match.group().replace('_', ' ').lower().strip()
            # Convert to number using word2number
            number = w2n.word_to_num(word)
            return str(number)
        except ValueError:
            # Return original text if conversion fails
            return match.group()
    
    # Create pattern for number words
    # This covers numbers up to trillion
    number_words = r'\b(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion|trillion)(?:\s+(?:and\s+)?(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion|trillion))*\b'
    # Convert number words to digits
    text = re.sub(number_words, word_to_number, text, flags=re.IGNORECASE)
    
    return text

In [6]:
csv_file = "../top_format_results.csv"
total_score = 0
num_entries = 0
max_lines_to_read =  847 # Define how many lines you want to process

# Read the CSV and calculate accuracy
with open(csv_file, mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    
    for i, row in enumerate(reader):
        if i >= max_lines_to_read:  # Stop after reading the desired number of lines
            break

        top_pred_string = row["output"]  # Extract the `output` column
        # Ensure the EXR string exists for this index
        if i < len(training_exr):
            exr_string = training_top[i]
            result = is_semantics_only_unordered_exact_match_post_ER_top_top(top_pred_string, exr_string, resolver)
            if(result!=1):
                print(top_pred_string)
                print(f"Row {i} - {result}")
            total_score += result
            num_entries += 1
        else:
            print(f"No corresponding EXR string for row {i}. Skipping.")

# Calculate average accuracy
accuracy = total_score / num_entries if num_entries > 0 else 0

# Print the result
print(f"Accuracy: {accuracy * 100:.2f}%")


(ORDER (PIZZAORDER (NUMBER two ) (SIZE medium ) (TOPPING sausage ) (TOPPING black ) (TOPPING olive ) ) (PIZZAORDER (NUMBER two ) (SIZE medium ) (TOPPING pepperoni ) (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) (PIZZAORDER (NUMBER three ) (SIZE large ) (TOPPING pepperoni ) (TOPPING sausage ) ) )
Row 0 - 0.8955223880597014
(ORDER (PIZZAORDER (NUMBER one ) (SIZE large ) (STYLE vegetarian ) (TOPPING pepper ) (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING banana ) ) ) )
Row 2 - 0.7741935483870969
(ORDER (PIZZAORDER (NUMBER one ) (SIZE large ) (TOPPING ham ) (TOPPING bacon ) (TOPPING onion ) (TOPPING black ) (TOPPING olive ) ) (PIZZAORDER (NUMBER one ) (SIZE medium ) (TOPPING sausage ) (TOPPING onion ) ) (DRINKORDER (NUMBER six ) (SIZE large ) (DRINKTYPE coke ) ) (DRINKORDER (NUMBER six ) (SIZE large ) (DRINKTYPE coke ) ) )
Row 5 - 0.7777777777777777
(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING ham ) (TOPPING grind ) (TOPPING beef ) (TOPPING pepperoni ) (TOPPING sausage ) (